## Apartado 1 - Clasificación de texto según el autor
Construid dos modelos de diferentes autores: Josep Carner y Miquel dels Sants Oliver. Para ello, podéis usar sus obras disponibles en el Proyecto Gutenberg. Por ejemplo, de Carner, tomad la traducción de los cuentos de Mark Twain. Luego, clasificad frases en el estilo de cada uno que muestren cómo vuestro modelo las identifica correctamente.

In [7]:
from my_utils import open_data
from text import *

hostal = open_data("CA-text/hostal.txt").read()
wordseq = words(hostal)

P_Miquel = UnigramWordModel(wordseq, 5)

adventures = open_data("CA-text/adventures.txt").read()
wordseq = words(adventures)

P_Josep = UnigramWordModel(wordseq, 5)

In [9]:
from learning import NaiveBayesLearner

dist = {('Miquel', 1): P_Miquel, ('Josep', 1): P_Josep}

nBS = NaiveBayesLearner(dist, simple=True)

In [10]:
def recognize(sentence, nBS):
    sentence = sentence.lower()
    sentence_words = words(sentence)
    
    return nBS(sentence_words)

Primero he probado a usar una frase extraída de los archivos CA-text/hostal.txt y CA-text/adventures.txt para comprobar si el modelo es capaz de identificar correctamente el autor de la frase.

In [20]:
recognize("havia donat la direcció a un personatge destinat a fer molt de renou", nBS)

'Miquel'

In [23]:
recognize("potser mai no bastarà mentre el món sigui món", nBS)

'Josep'

Para probar con otro ejemplo, le he preguntado al ChatGPT la diferencia de estilo entre los dos autores. A continuación, se muestra la respuesta del ChatGPT:

```
Diferencias resumidas:
	•	Josep Carner → Poètic, refinat, musical i idealitzat.
	•	Miquel dels Sants Oliver → Narratiu, directe, reflexiu i analític.
```

A raíz de esto he probado a clasificar dos frases de cada autor para ver si el modelo es capaz de diferenciar el estilo de cada uno. Pero en este caso estas frases no han sido sacadas de los archivios .txt sino generadas.

In [19]:
recognize("Els carrers, encara humits, semblaven desertar de la nit", nBS)

'Miquel'

In [17]:
recognize("Amb un gest majestuós em fità un instant", nBS)

'Josep'